In [1]:
import numpy as np
import tensorflow as tf

batch_size = 128;embedding_dimension = 64;num_classes = 2
hidden_layer_size = 32;times_steps = 6;element_size = 1

digit_to_word_map = {1:"One",2:"Two", 3:"Three", 4:"Four", 5:"Five",6:"Six",7:"Seven",8:"Eight",9:"Nine"}
digit_to_word_map[0] = "PAD"
even_sentences = []
odd_sentences = []
seqlens = []
for i in range(10000):
    rand_seq_len = np.random.choice(range(3,7))
    seqlens.append(rand_seq_len)
    rand_odd_ints = np.random.choice(range(1,10,2), rand_seq_len)
    rand_even_ints = np.random.choice(range(2,10,2),rand_seq_len)
    if rand_seq_len<6:
        rand_odd_ints = np.append(rand_odd_ints, [0]*(6-rand_seq_len))
        rand_even_ints = np.append(rand_even_ints, [0]*(6-rand_seq_len))
    even_sentences.append(" ".join([digit_to_word_map[r] for r in rand_odd_ints]))
    odd_sentences.append(" ".join([digit_to_word_map[r] for r in rand_even_ints]))

data = even_sentences+odd_sentences
# Same seq lengths for even, odd sentences
seqlens*=2

even_sentences[0:6]

word2index_map ={}
index=0
for sent in data:
    for word in sent.lower().split():
        if word not in word2index_map:
            word2index_map[word] = index
            index+=1
# Inverse map
index2word_map = {index: word for word, index in word2index_map.items()}
vocabulary_size = len(index2word_map)

labels = [1]*10000 + [0]*10000
for i in range(len(labels)):
    label = labels[i]
    one_hot_encoding = [0]*2
    one_hot_encoding[label] = 1
    labels[i] = one_hot_encoding

data_indices = list(range(len(data)))
np.random.shuffle(data_indices) #shuffle
data = np.array(data)[data_indices]
labels = np.array(labels)[data_indices]
seqlens = np.array(seqlens)[data_indices]
train_x = data[:10000]
train_y = labels[:10000]
train_seqlens = seqlens[:10000]

test_x = data[10000:]
test_y = labels[10000:]
test_seqlens = seqlens[10000:]

def get_sentence_batch(batch_size,data_x, data_y,data_seqlens):
    instance_indices = list(range(len(data_x)))
    np.random.shuffle(instance_indices)
    batch = instance_indices[:batch_size]
    x = [[word2index_map[word] for word in data_x[i].lower().split()] for i in batch]
    y = [data_y[i] for i in batch]
    seqlens = [data_seqlens[i] for i in batch]
    return x,y,seqlens

_inputs = tf.placeholder(tf.int32, shape=[batch_size,times_steps])
_labels = tf.placeholder(tf.float32, shape=[batch_size, num_classes])
# seqlens for dynamic calculation
_seqlens = tf.placeholder(tf.int32, shape=[batch_size])

with tf.name_scope("embeddings"):
    embeddings = tf.Variable(tf.random_uniform([vocabulary_size,embedding_dimension],-1.0, 1.0),name='embedding')
    embed = tf.nn.embedding_lookup(embeddings, _inputs)


#LSTM
with tf.variable_scope("lstm"):
    lstm_cell = tf.contrib.rnn.BasicLSTMCell(hidden_layer_size,forget_bias=1.0)
    outputs, states = tf.nn.dynamic_rnn(lstm_cell, embed,sequence_length = _seqlens, dtype=tf.float32)
    weights = { 'linear_layer': tf.Variable(tf.truncated_normal([hidden_layer_size,num_classes],mean=0,stddev=.01))}
    biases ={'linear_layer':tf.Variable(tf.truncated_normal([num_classes],mean=0,stddev=.01))}
    # Extract the last relevant output and use in a linear layer
    final_output = tf.matmul(states[1],weights["linear_layer"]) + biases["linear_layer"]
    softmax = tf.nn.softmax_cross_entropy_with_logits(logits = final_output,labels = _labels)
    cross_entropy = tf.reduce_mean(softmax)
    
train_step = tf.train.RMSPropOptimizer(0.001, 0.9).minimize(cross_entropy)
correct_prediction = tf.equal(tf.argmax(_labels,1), tf.argmax(final_output,1))
accuracy = (tf.reduce_mean(tf.cast(correct_prediction,tf.float32)))*100
with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())
    for step in range(1000):
        x_batch, y_batch,seqlen_batch = get_sentence_batch(batch_size,train_x,train_y,train_seqlens)
        sess.run(train_step,feed_dict={_inputs:x_batch, _labels:y_batch, _seqlens:seqlen_batch})
        if step % 100 == 0:
            acc = sess.run(accuracy,feed_dict={_inputs:x_batch,_labels:y_batch,_seqlens:seqlen_batch})
            print("Accuracy at %d: %.5f" % (step, acc))
    for test_batch in range(5):
        x_test, y_test,seqlen_test = get_sentence_batch(batch_size,test_x,test_y,test_seqlens)
        batch_pred,batch_acc = sess.run([tf.argmax(final_output,1),accuracy],
        feed_dict={_inputs:x_test,_labels:y_test,_seqlens:seqlen_test})
        print("Test batch accuracy %d: %.5f" % (test_batch, batch_acc))
    output_example = sess.run([outputs],feed_dict={_inputs:x_test,_labels:y_test,_seqlens:seqlen_test})
    states_example = sess.run([states[1]],feed_dict={_inputs:x_test,_labels:y_test,_seqlens:seqlen_test})
    states_example1 = sess.run([states],feed_dict={_inputs:x_test,_labels:y_test,_seqlens:seqlen_test})


Instructions for updating:

Future major versions of TensorFlow will allow gradients to flow
into the labels input on backprop by default.

See @{tf.nn.softmax_cross_entropy_with_logits_v2}.

Accuracy at 0: 73.43750
Accuracy at 100: 100.00000
Accuracy at 200: 100.00000
Accuracy at 300: 100.00000
Accuracy at 400: 100.00000
Accuracy at 500: 100.00000
Accuracy at 600: 100.00000
Accuracy at 700: 100.00000
Accuracy at 800: 100.00000
Accuracy at 900: 100.00000
Test batch accuracy 0: 100.00000
Test batch accuracy 1: 100.00000
Test batch accuracy 2: 100.00000
Test batch accuracy 3: 100.00000
Test batch accuracy 4: 100.00000


In [40]:
output_example[0][1] #第2个句子的y1,y2,y3,y4,y5,y6 ,即对应的state 1 -6 的各个神经元参数

array([[ 0.27944392, -0.54158884, -0.58350223,  0.4804586 , -0.51977104,
        -0.4177508 ,  0.3554054 , -0.45807445, -0.52155864,  0.52196264,
         0.47510675, -0.45088238,  0.45196566, -0.4896564 , -0.51467115,
        -0.40138274,  0.614623  , -0.40318283,  0.2357543 , -0.32615232,
        -0.48341882,  0.377625  , -0.40499815,  0.30868793,  0.4935338 ,
        -0.42156386,  0.22172411, -0.14312477,  0.45683843, -0.29186282,
        -0.46392113,  0.25222215],
       [ 0.62624604, -0.7989841 , -0.82225066,  0.7441918 , -0.80905545,
        -0.7405462 ,  0.6765096 , -0.7301297 , -0.8032071 ,  0.794739  ,
         0.7637833 , -0.7749163 ,  0.73039556, -0.7661446 , -0.7774762 ,
        -0.69653213,  0.8731955 , -0.6580318 ,  0.5824946 , -0.5979315 ,
        -0.7542486 ,  0.74311113, -0.721142  ,  0.54478663,  0.77420825,
        -0.7428881 ,  0.505696  , -0.29532123,  0.7522843 , -0.5963745 ,
        -0.76621604,  0.4892354 ],
       [ 0.8107909 , -0.78188735, -0.85633326,  0.8601

In [3]:
states_example[0][1]


array([ 0.8244984 , -0.8837261 , -0.9164325 ,  0.88607794, -0.9323245 ,
       -0.8979708 ,  0.869714  , -0.87029904, -0.92366487,  0.9232877 ,
        0.89592713, -0.9102634 ,  0.88085425, -0.8548438 , -0.8665261 ,
       -0.8429517 ,  0.964144  , -0.7791912 ,  0.7748821 , -0.7753577 ,
       -0.88919324,  0.92110074, -0.87192583,  0.6893033 ,  0.8870727 ,
       -0.87398255,  0.70223844, -0.35227987,  0.9078322 , -0.8290536 ,
       -0.9023689 ,  0.58018273], dtype=float32)

In [12]:
states_example1 #有两个隐状态c,和h， c是输入门, h是输出门
                #state是final state，如果有n layer，则是final state也有n个元素，对应每一层的state。 

[LSTMStateTuple(c=array([[-1.4841735,  1.7360431,  1.405453 , ...,  1.5869547,  1.2460747,
         -2.05087  ],
        [ 3.0395713, -3.4637344, -3.472891 , ..., -2.4197836, -3.2415657,
          2.3022718],
        [-1.7928432,  1.775701 ,  1.8385878, ...,  1.8309517,  1.606359 ,
         -1.9575055],
        ...,
        [ 2.2311816, -2.1150064, -2.3189468, ..., -2.015119 , -2.335775 ,
          1.7197986],
        [ 2.6387937, -2.5974407, -2.8085098, ..., -2.4167647, -2.8322644,
          2.116045 ],
        [ 2.6751957, -2.4151773, -2.8195524, ..., -2.3012977, -2.7054276,
          1.9294608]], dtype=float32), h=array([[-0.77786416,  0.81522083,  0.68201816, ...,  0.8207027 ,
          0.6731336 , -0.8942009 ],
        [ 0.8244984 , -0.8837261 , -0.9164325 , ..., -0.8290536 ,
         -0.9023689 ,  0.58018273],
        [-0.7682362 ,  0.8482324 ,  0.8112685 , ...,  0.79402673,
          0.6098539 , -0.8318191 ],
        ...,
        [ 0.86157584, -0.7911887 , -0.8758014 , ..., -0.8

In [41]:
states_example1

IndexError: list index out of range